In [1]:
!pip install keras-tuner -q
!pip install lightgbm -q

     |████████████████████████████████| 97 kB 2.4 MB/s 


### Load the California housing price prediction dataset

In [2]:
from sklearn.datasets import fetch_california_housing

house_dataset = fetch_california_housing()

# Import pandas package to format the data
import pandas as pd

# Extract features with their names into the a dataframe format
data = pd.DataFrame(house_dataset.data, columns=house_dataset.feature_names)

# Extract target with their names into a pd.Series object with name MEDV
target = pd.Series(house_dataset.target, name = 'MEDV')

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, shuffle=False)


### Use LightGBM GBDT model to do regression

In [3]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error

gbdt_model = lgb.LGBMRegressor(
                    boosting_type='gbdt',
                    num_leaves=31,
                    learning_rate=0.05,
                    n_estimators=10
                    )  # create model

validation_data = (X_val, y_val)
gbdt_model.fit(X_train, y_train,
        eval_set=[validation_data],
        eval_metric='mse',
        early_stopping_rounds=5) # fit the model

# evalaute model
y_pred_gbdt = gbdt_model.predict(X_test, num_iteration=gbdt_model.best_iteration_)
test_mse_1 = mean_squared_error(y_test, y_pred_gbdt)
print("The GBDT prediction MSE on test set: {}".format(test_mse_1))

# save, load, and evaluate the model
fname = 'gbdt_model.txt'
gbdt_model.booster_.save_model(fname, num_iteration=gbdt_model.best_iteration_) 

gbdt_model_2 = lgb.Booster(model_file=fname)
gbdt_model_2.predict(X_test)
test_mse_2 = mean_squared_error(y_test, y_pred_gbdt)
print("The reloaded GBDT prediction MSE on test set: {}".format(test_mse_2))


[1]	valid_0's l2: 1.28051	valid_0's l2: 1.28051
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's l2: 1.2009	valid_0's l2: 1.2009
[3]	valid_0's l2: 1.1316	valid_0's l2: 1.1316
[4]	valid_0's l2: 1.06506	valid_0's l2: 1.06506
[5]	valid_0's l2: 1.00734	valid_0's l2: 1.00734
[6]	valid_0's l2: 0.952642	valid_0's l2: 0.952642
[7]	valid_0's l2: 0.903993	valid_0's l2: 0.903993
[8]	valid_0's l2: 0.857114	valid_0's l2: 0.857114
[9]	valid_0's l2: 0.815805	valid_0's l2: 0.815805
[10]	valid_0's l2: 0.778275	valid_0's l2: 0.778275
Did not meet early stopping. Best iteration is:
[10]	valid_0's l2: 0.778275	valid_0's l2: 0.778275
The GBDT prediction MSE on test set: 0.7514642734431766
The reloaded GBDT prediction MSE on test set: 0.7514642734431766


### Create the LightGBM model building function

In [4]:
def build_model(hp):
    model = lgb.LGBMRegressor(
                        boosting_type='gbdt',
                        num_leaves=hp.Choice("num_leaves", [15, 31, 63], default=31),
                        learning_rate=hp.Float('learning_rate', 1e-3, 10, sampling='log', default=0.05),
                        n_estimators=hp.Int('n_estimators', 10, 200, step=10)
                        )

    return model

### Customize the LightGBM tuner

In [5]:
import os
import pickle
import tensorflow as tf
import kerastuner as kt
import lightgbm as lgb
from sklearn.metrics import mean_squared_error

class LightGBMTuner(kt.engine.base_tuner.BaseTuner):

    def run_trial(self, trial, X, y, validation_data):
        model = self.hypermodel.build(trial.hyperparameters) # build the model
        model.fit(X_train, y_train,
                eval_set=[validation_data],
                eval_metric='mse',
                early_stopping_rounds=5) # fit the model
        X_val, y_val = validation_data
        y_pred = model.predict(X_val, num_iteration=model.best_iteration_) # evaluate the model
        eval_mse = mean_squared_error(y_val, y_pred)
        self.oracle.update_trial(trial.trial_id, {'mse': eval_mse})  # inform the oracle of the eval result, the result is a dictionary with the metric names as the keys.
        self.save_model(trial.trial_id, model) # save the model to disk

    def save_model(self, trial_id, model, step=0):
        fname = os.path.join(self.get_trial_dir(trial_id), 'model.txt')
        model.booster_.save_model(fname, num_iteration=model.best_iteration_) 

    def load_model(self, trial):
        fname = os.path.join(self.get_trial_dir(trial.trial_id), 'model.txt')
        model = lgb.Booster(model_file=fname)
        return model

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  after removing the cwd from sys.path.


### Run the tuner to select a LightGBM models for the housing price prediction

In [6]:
my_lightgbm_tuner = LightGBMTuner(
        oracle=kt.oracles.RandomSearch(
            objective=kt.Objective('mse', 'min'),
            max_trials=10,
            seed=42),
        hypermodel=build_model,
        overwrite=True,
        project_name='my_lightgbm_tuner')

my_lightgbm_tuner.search(X_train, y_train, validation_data=(X_val, y_val))

Trial 10 Complete [00h 00m 00s]
mse: 11.151438308866505

Best mse So Far: 0.2202899505068673
Total elapsed time: 00h 00m 14s
INFO:tensorflow:Oracle triggered exit


### Evaluate the best discovered model

In [7]:
from sklearn.metrics import mean_squared_error

best_model = my_lightgbm_tuner.get_best_models(1)[0]
y_pred_test = best_model.predict(X_test)
test_mse = mean_squared_error(y_test, y_pred_test)
print("The prediction MSE on test set: {}".format(test_mse))


The prediction MSE on test set: 0.20391543433512713


### Check the best model

In [8]:
my_lightgbm_tuner.get_best_models(1)

In [9]:
my_lightgbm_tuner.results_summary(1)

Results summary
Results in ./my_lightgbm_tuner
Showing 1 best trials
Objective(name='mse', direction='min')
Trial summary
Hyperparameters:
num_leaves: 31
learning_rate: 0.09504947970741313
n_estimators: 190
Score: 0.2202899505068673
